In [ ]:
include("../Polyhedron.jl")
include("interlocking.jl")
include("../plotting.jl")
include("../decomposition.jl")

In [ ]:
@time facet_adjacency(Candy)

In [ ]:
reverse!(Candy.facets[1])

In [ ]:
@time adjfacets(Candy, get_facets(Candy)[1])

In [ ]:
@time orient_facets!(Candy)

In [ ]:
@time map(f -> adjfacets(block, f, check = false), get_facets(block))

### Showcase of Interlocking test

In [ ]:
ass, frame = assembly1()

display(plot(ass, width = 360, height = 360))

titest(ass, frame)

### non periodic interlocking

In [ ]:
function read_vertices_faces(filename)
    vertices = Vector{Vector{Float64}}()
    blocks = Vector{Vector{Vector{Int}}}()
    
    open(filename, "r") do file
        last = "0"
        block = Vector{Vector{Int}}()
        for line in eachline(file)
            parts = split(line)
            if isempty(parts)
                continue
            end
            if parts[1] == "v"
                v = parse.(Float64, parts[2:end])
                push!(vertices, v)
                last = "v"
            elseif parts[1] == "f"
                if last == "v" && length(block) > 0
                    push!(blocks, block)
                    block = Vector{Vector{Int}}()
                end
                f = parse.(Int, parts[2:end])
                push!(block, f)
                last = "f"
            end
        end
        push!(blocks, block)
    end
    
    return vertices, blocks
end

function read_frame(filename)
    frame = Vector{Int}()
    open(filename, "r") do file
        for line in eachline(file)
            push!(frame, parse(Int, line))
        end
    end
    return frame
end

In [ ]:
function indexmap(block::Vector{<:Vector{<:Integer}})
    verts = sort(unique(vcat(block...)))
    return i -> findfirst(v -> v == i, verts)
end

function assembly_from_file(ass_file, frame_file)
    v, blocks = read_vertices_faces(ass_file)
    vertices_of_blocks = map(b -> v[sort(unique(vcat(b...)))], blocks)
    facets_of_blocks = map(b -> [indexmap(b).(f) for f in b], blocks)
    ass = [Polyhedron(verts = vertices_of_blocks[i], facets = facets_of_blocks[i]) for i in eachindex(blocks)]
    frame = read_frame(frame_file).+1

    return ass, frame
end

In [ ]:
include("../Polyhedron.jl")

In [ ]:
@time assembly, fr = assembly_from_file("9.obj", "frame_9.txt");

In [ ]:
display(ass[1])

In [ ]:
plot(ass)

In [ ]:
titest(assembly, fr)

In [ ]:
include("interlocking.jl")
include("../plotting.jl")

In [ ]:
include("[-05,05].jl")
ass = [Polyhedron(facets = faces[i], verts = coordinates[i]) for i in eachindex(faces)]
frame = collect(2:9)

framecolor = RGB(0.,0.,0.)
color = [RGB(1.,0.,0.), RGB(0.,1.,0.), RGB(0.,0.,1.), RGB(0.5,0,0.5), RGB(0.5,0.5,0), RGB(0,0.5,0.5), RGB(0.5, 0.25,0.25), RGB(0.25,0.5,0.25), RGB(0.25,0.25,0.5)]

inds = [1,2,3,4,5,6,7,8,9]
assembly = ass[inds]
fr = Int[]

display(plot(assembly, facetcolors = map(i -> i in fr ? [framecolor] : [color[i]], eachindex(assembly)), opacity = 1))

try
    model = wangtest(assembly, fr)
    # write(io, model)
catch e
    showerror(e)
end

## Meshes experiments

In [ ]:
using Meshes

In [ ]:
block1 = flattenfacets(block)

In [ ]:
face = (get_facets(block1)[end-1])

In [ ]:
@time(orient_facets_ccw!(ass_9[2]))

In [ ]:
M = get_verts(block1)

In [ ]:
@time polyareas = map(f -> PolyArea(map(i->Tuple(M[:, i]), f)), get_facets(block1))

In [ ]:
reverse!(block1.facets[1])

In [ ]:
@time orientation.(polyareas)

In [ ]:
orientation(polyarea)

In [ ]:
@time mesh = discretize(polyarea, Dehn1899())

In [ ]:
adjacencymatrix(mesh.topology)

In [ ]:
Vector(coordinates(mesh.vertices[1]))

In [ ]:
Polyhedron(Point([0,0,0]...), Point([1,0,0]...), Point([0,1,0]...), Point([0,0,1]...))

In [ ]:
typeof(Tetrahedron((0,0,0),(1,0,0),(0,1,0),(0,0,1)))

In [ ]:
Tetrahedron([0,0,0], [1,0,0], [0,1,0], [0,0,1])

In [ ]:
intersection(polyarea, polyarea)

In [ ]:
include("../Polyhedron.jl")

In [ ]:
@time facet_adjacency(block1)

In [ ]:
@time vol(block1, is_oriented = true)

In [ ]:
@time orient_facets(block1)